In [1]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: sample_submission.csv   
  inflating: data_description.txt    
  inflating: test.csv                
  inflating: train.csv               


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

In [0]:
'''
データのロード
'''
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
pd.set_option('display.max_columns', 90)

In [8]:
train_data.loc[:,train_data.columns[::-1]].head()

,SalePrice,SaleCondition,SaleType,YrSold,MoSold,MiscVal,MiscFeature,Fence,PoolQC,PoolArea,ScreenPorch,3SsnPorch,EnclosedPorch,OpenPorchSF,WoodDeckSF,PavedDrive,GarageCond,GarageQual,GarageArea,GarageCars,GarageFinish,GarageYrBlt,GarageType,FireplaceQu,Fireplaces,Functional,TotRmsAbvGrd,KitchenQual,KitchenAbvGr,BedroomAbvGr,HalfBath,FullBath,BsmtHalfBath,BsmtFullBath,GrLivArea,LowQualFinSF,2ndFlrSF,1stFlrSF,Electrical,CentralAir,HeatingQC,Heating,TotalBsmtSF,BsmtUnfSF,BsmtFinSF2,BsmtFinType2,BsmtFinSF1,BsmtFinType1,BsmtExposure,BsmtCond,BsmtQual,Foundation,ExterCond,ExterQual,MasVnrArea,MasVnrType,Exterior2nd,Exterior1st,RoofMatl,RoofStyle,YearRemodAdd,YearBuilt,OverallCond,OverallQual,HouseStyle,BldgType,Condition2,Condition1,Neighborhood,LandSlope,LotConfig,Utilities,LandContour,LotShape,Alley,Street,LotArea,LotFrontage,MSZoning,MSSubClass,Id
0,208500,Normal,WD,2008,2,0,NaN,NaN,NaN,0,0,0,0,61,0,Y,TA,TA,548,2,RFn,2003.0,Attchd,NaN,0,Typ,8,Gd,1,3,1,2,0,1,1710,0,854,856,SBrkr,Y,Ex,GasA,856,150,0,Unf,706,GLQ,No,TA,Gd,PConc,TA,Gd,196.0,BrkFace,VinylSd,VinylSd,CompShg,Gable,2003,2003,5,7,2Story,1Fam,Norm,Norm,CollgCr,Gtl,Inside,AllPub,Lvl,Reg,NaN,Pave,8450,65.0,RL,60,1
1,181500,Normal,WD,2007,5,0,NaN,NaN,NaN,0,0,0,0,0,298,Y,TA,TA,460,2,RFn,1976.0,Attchd,TA,1,Typ,6,TA,1,3,0,2,1,0,1262,0,0,1262,SBrkr,Y,Ex,GasA,1262,284,0,Unf,978,ALQ,Gd,TA,Gd,CBlock,TA,TA,0.0,None,MetalSd,MetalSd,CompShg,Gable,1976,1976,8,6,1Story,1Fam,Norm,Feedr,Veenker,Gtl,FR2,AllPub,Lvl,Reg,NaN,Pave,9600,80.0,RL,20,2
2,223500,Normal,WD,2008,9,0,NaN,NaN,NaN,0,0,0,0,42,0,Y,TA,TA,608,2,RFn,2001.0,Attchd,TA,1,Typ,6,Gd,1,3,1,2,0,1,1786,0,866,920,SBrkr,Y,Ex,GasA,920,434,0,Unf,486,GLQ,Mn,TA,Gd,PConc,TA,Gd,162.0,BrkFace,VinylSd,VinylSd,CompShg,Gable,2002,2001,5,7,2Story,1Fam,Norm,Norm,CollgCr,Gtl,Inside,AllPub,Lvl,IR1,NaN,Pave,11250,68.0,RL,60,3
3,140000,Abnorml,WD,2006,2,0,NaN,NaN,NaN,0,0,0,272,35,0,Y,TA,TA,642,3,Unf,1998.0,Detchd,Gd,1,Typ,7,Gd,1,3,0,1,0,1,1717,0,756,961,SBrkr,Y,Gd,GasA,756,540,0,Unf,216,ALQ,No,Gd,TA,BrkTil,TA,TA,0.0,None,Wd Shng,Wd Sdng,CompShg,Gable,1970,1915,5,7,2Story,1Fam,Norm,Norm,Crawfor,Gtl,Corner,AllPub,Lvl,IR1,NaN,Pave,9550,60.0,RL,70,4
4,250000,Normal,WD,2008,12,0,NaN,NaN,NaN,0,0,0,0,84,192,Y,TA,TA,836,3,RFn,2000.0,Attchd,TA,1,Typ,9,Gd,1,4,1,2,0,1,2198,0,1053,1145,SBrkr,Y,Ex,GasA,1145,490,0,Unf,655,GLQ,Av,TA,Gd,PConc,TA,Gd,350.0,BrkFace,VinylSd,VinylSd,CompShg,Gable,2000,2000,5,8,2Story,1Fam,Norm,Norm,NoRidge,Gtl,FR2,AllPub,Lvl,IR1,NaN,Pave,14260,84.0,RL,60,5


In [0]:
'''
クリーニング
'''
# object型のコラムを抜き出し
def get_cat_cols(df):
  return [col for col in df.columns if df[col].dtype == 'object']

In [0]:
y = np.log1p(train_data.SalePrice)

In [0]:
cand_train_predictors = train_data.drop(['Id', 'SalePrice'], axis=1)
cand_test_predictors = test_data.drop(['Id'], axis=1)

In [0]:
cat_cols = get_cat_cols(cand_train_predictors)

In [0]:
cand_train_predictors[cat_cols] = cand_train_predictors[cat_cols].fillna('NotAvailable')
cand_test_predictors[cat_cols] = cand_test_predictors[cat_cols].fillna('NotAvailable')

In [0]:
# エンコード
encoders = {}
for col in cat_cols:
    encoders[col] = LabelEncoder()
    val = cand_train_predictors[col].tolist()
    val.extend(cand_test_predictors[col].tolist())
    encoders[col].fit(val)
    cand_train_predictors[col] = encoders[col].transform(cand_train_predictors[col])+1
    cand_test_predictors[col] = encoders[col].transform(cand_test_predictors[col])+1

In [0]:
'''
特徴量の選択
'''
corr_matrix = cand_train_predictors.corr().abs()   # 相関係数の絶対値の計算
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))   # corr_matrixの上三角成分を抽出
cols_to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print('Highly correlated features(will be droped):',cols_to_drop)

Highly correlated features(will be droped): ['Exterior2nd', '1stFlrSF', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea', 'PoolQC']


In [0]:
cand_train_predictors_droped = cand_train_predictors.drop(cols_to_drop, axis=1)
cand_test_predictors_droped = cand_test_predictors.drop(cols_to_drop, axis=1)

In [0]:
train_set, test_set = cand_train_predictors_droped.align(cand_test_predictors_droped,join='left', axis=1)
# train_set = np.log1p(train_set)
# test_set = np.log1p(test_set)

In [0]:
for col in train_set:
  if train_set[col].skew() > 0.6:
    train_set[col] = np.log1p(train_set[col])
    test_set[col] = np.log1p(test_set[col])

In [0]:
for col in train_set:
  if train_set[col].skew() > 0.6:
    print(col)

In [0]:
'''
モデル
'''
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error

In [0]:
regressor = XGBRegressor(
    n_estimators=1000, 
    random_state=1,
    learning_rate=0.07300000000000001
)
my_model = regressor
kf = KFold(5, shuffle=True, random_state=1).get_n_splits(train_set)
rmse = np.sqrt(-cross_val_score(my_model, train_set,y, scoring="neg_mean_squared_error", cv=kf, verbose=0))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
my_model.fit(train_set.values, y)
print('-'*80)
print(my_model.score(train_set.values,y))
print('rmse:',rmse.mean())
train_pred = my_model.predict(train_set.values)
print('rmsle: ',np.sqrt(mean_squared_log_error(y, train_pred)))
print('mae: ',mean_absolute_error(train_data.SalePrice, np.expm1(train_pred)))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


--------------------------------------------------------------------------------
0.9919747380385158
rmse: 0.12658898844704622
rmsle:  0.0027719453135066547
mae:  4566.376340432363


In [0]:
'''
提出
'''
predicted_prices = np.expm1(my_model.predict(test_set.values))
print(predicted_prices[:5])

my_submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predicted_prices})
my_submission.Id = my_submission.Id.astype(int)
my_submission.to_csv('submission.csv', index=False)

[116392.055 163346.08  190746.84  195655.78  189295.47 ]


In [0]:
'''
データの保存
'''
import pickle
Id = test_data.Id

pickle_file = "./HousePrice_data_kernel.pickle"
with open(pickle_file, 'wb') as f:
    pickle.dump(train_set, f)
    pickle.dump(y, f)
    pickle.dump(test_set, f)
    pickle.dump(train_data, f)
    pickle.dump(test_data, f)
    pickle.dump(Id, f)

In [0]:
with open(pickle_file, 'rb') as f:
    X_train = pickle.load(f)
    Y_train = pickle.load(f)
    X_test = pickle.load(f)
    train = pickle.load(f)
    test = pickle.load(f)
    Id = pickle.load(f)

In [0]:
X_train.shape

(1460, 73)